<a href="https://colab.research.google.com/github/riyakb/Relevant-Answer-Search-Engine/blob/master/ELMO_and_deep_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import keras
from keras.utils import np_utils
from keras.models import Model

activation_fn = 'elu'

########################### DEFINE YOUR DL MODEL HERE ############################################

def nn_model(query_input_len, answer_input_len, batch_size):
  ############### for query input #########################
  
  layer1_1 = keras.layers.Input(shape=(query_input_len, embedding_dim, 1))
  conv1_1 = keras.layers.convolutional.Conv2D(32, (2, 5), activation=activation_fn)(layer1_1)
  pool1_1 = keras.layers.convolutional.AveragePooling2D()(conv1_1)
  conv1_2 = keras.layers.convolutional.Conv2D(32, (2, 5), activation=activation_fn)(pool1_1)
  pool1_2 = keras.layers.convolutional.AveragePooling2D()(conv1_2)
  flat1_1 = keras.layers.Flatten()(pool1_2)
  drop1_1 = keras.layers.Dropout(0.24)(flat1_1)
  dense1_1 = keras.layers.Dense(800, activation=activation_fn)(drop1_1)
  drop1_2 = keras.layers.Dropout(0.28)(dense1_1)
  dense1_2 = keras.layers.Dense(500, activation=activation_fn)(drop1_1)
  drop1_3 = keras.layers.Dropout(0.25)(dense1_2)
  
  
  ############### For answers input #######################

  layer2_1 = keras.layers.Input(shape=(answer_input_len, embedding_dim, 1))
  conv2_1 = keras.layers.convolutional.Conv2D(32, (2, 5), activation=activation_fn)(layer2_1)
  pool2_1 = keras.layers.convolutional.AveragePooling2D()(conv2_1)
  conv2_2 = keras.layers.convolutional.Conv2D(32, (2, 5), activation=activation_fn)(pool2_1)
  pool2_2 = keras.layers.convolutional.AveragePooling2D()(conv2_2)
  conv2_2 = keras.layers.convolutional.Conv2D(32, (2, 5), activation=activation_fn)(pool2_2)
  pool2_3 = keras.layers.convolutional.AveragePooling2D()(conv2_2)
  flat2_1 = keras.layers.Flatten()(pool2_3)
  drop2_1 = keras.layers.Dropout(0.1)(flat2_1)
  dense2_1 = keras.layers.Dense(3000, activation=activation_fn)(drop2_1)
  drop2_2 = keras.layers.Dropout(0.3)(dense2_1)
  dense2_2 = keras.layers.Dense(1500, activation=activation_fn)(drop2_2)
  drop2_3 = keras.layers.Dropout(0.2)(dense2_2)
  dense2_3 = keras.layers.Dense(500, activation=activation_fn)(drop2_3)
  drop2_4 = keras.layers.Dropout(0.2)(dense2_3)
#   dense2_4 = keras.layers.Dense(500, activation=activation_fn)(drop2_4)
#   drop2_5 = keras.layers.Dropout(0.2)(dense2_4)
#   dense2_5 = keras.layers.Dense(200, activation=activation_fn)(drop2_5)
#   drop2_6 = keras.layers.Dropout(0.2)(dense2_5)
  
  ############## Final merge of 2 models ##################
  
  merge = keras.layers.multiply([drop1_3, drop2_4])
  
  final_l1 = keras.layers.Dense(200, activation=activation_fn)(merge)
#   final_l2 = keras.layers.Dense(100, activation=activation_fn)(final_l1)
  final_l3 = keras.layers.Dense(10, activation=activation_fn)(final_l1)
  final = keras.layers.Dense(1, activation='sigmoid')(final_l3)
  
  final_model = Model(inputs=[layer1_1, layer2_1], outputs=final)
  
  return final_model

##################################################################################################

In [0]:

model = nn_model(max_query_len, max_answer_len, batch_size)
model.compile('Adadelta', 'binary_crossentropy')

count=0
break_point = 500

for  data in pd.read_csv('rish_data.tsv', sep='\t', chunksize=batch_size, names=['id', 'query', 'answers', 'correct answer', 'count']):
   
  count+=1
  
  query_Train = convert_data(data['query'], max_query_len)
  answer_Train = convert_data(data['answers'], max_answer_len)
  yTrain = data['correct answer']
  
#   print(query_Train.shape)
#   print(answer_Train.shape)
   
#   print(X['query'][0].shape)
#   print(X['answers'].shape)
  
  query_Train = query_Train.reshape(batch_size, max_query_len, embedding_dim, 1)
  answer_Train = answer_Train.reshape(batch_size, max_answer_len, embedding_dim, 1)
  
  ################### Training code over here, and function to save model also. Pickle library already imported ###################
  
  history = model.fit([query_Train, answer_Train], yTrain, validation_split=0.2, verbose=0, epochs=18)
  
  model.save("cnn_Model_1.h5")
  print(count)
    
  ########################################################################
  
# model.save("cnn_Model_"+str(count)+".h5")

In [0]:
# history.history

plt.plot(history.history['loss'], label = "Training Loss")
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='lower left')
plt.show()

In [0]:
!pip install allennlp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
import tensorflow_hub as hub
import tensorflow as tf
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
# from allennlp.commands.elmo import ElmoEmbedder
# elmo = ElmoEmbedder()

# dssm_scores = np.load('dssm_scores.npy')
# cdssm_scores = np.load('cdssm_scores.npy')

In [0]:

q_embed_save = np.load('q_embedding.npy')
a_embed_save = np.load('a_embedding.npy')

tf.logging.set_verbosity(tf.logging.ERROR)

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

count=0

firstRun=True

# q_embed_save=np.empty([0,0,0])
# a_embed_save=np.empty([0,0,0])

print('Starting execution of for loop')

for data in pd.read_csv("reduced_data.tsv", sep='\t', chunksize=100, names=['id', 'query', 'answers', 'correct answer', 'count']):
  data = data.reset_index()
  if data['index'][0]>100000:
    break
  elif q_embed_save.shape[0]>data['index'][0]:
    count=q_embed_save.shape[0]/100
    print(count)
    pass
  else:
#     q = []
#     a = []
#     for j in range(data.shape[0]):
#       q.append(" ".join(convert_string(data['query'][j])))
#       a.append(" ".join(convert_string(data['answers'][j])))
    q_embed = elmo(data['query'].tolist(), signature="default", as_dict=True)["elmo"]
    a_embed = elmo(data['answers'].tolist(), signature="default", as_dict=True)["elmo"]

    q_embed=sess.run(q_embed)
    a_embed=sess.run(a_embed)

    y_train = data['correct answer']

    q_embed_2 = pad_sequences(q_embed, maxlen = 20)
    a_embed_2 = pad_sequences(a_embed, maxlen = 150)

    if count==0:
      q_embed_save=np.copy(q_embed_2)
      a_embed_save=np.copy(a_embed_2)

    q_embed_save = np.vstack((q_embed_save, q_embed_2))
    a_embed_save = np.vstack((a_embed_save, a_embed_2))

  #   print(q_embed_2.shape)
  #   print(a_embed_2.shape)
  #   print(np.append(q_embed, a_embed, axis=0).shape)
    
    if count%20==0:
      np.save("q_embedding.npy", q_embed_save)
      np.save("a_embedding.npy", a_embed_save)
      print(count)
    count+=1
    
    
#     break

# print(np.vstack((q_embed_2, a_embed_2)).shape)
